## Practice OOP: a `Rock` class

This notebook provides more practice on classes.

We'll implement a class to represent a rock. We're focused on rock physics here, but feel free to implement other characteristics of the rock. For example:

- _Petrological:_ mineralogic composition, mineral proportions, photomicrograph, technical name, etc.
- _Geomechanical:_ fabric, hardness, brittleness, etc.
- _Sedimentological:_ grain size, grain size proportions (from sieve data), texture, structures, colour, etc.
- _Curatorial:_ rock type, id, provenance, collector, etc.

If you use one of these other facets, you'll need to come up with other methods to add. If you're unsure, talk to one of the instructors, or talk it over with a partner.

----

First, we'll define a very basic class. Then we'll add to that.

In [1]:
class Rock:
    def __init__(self, vp, rho=None):
        self.vp = vp
        self.rho = rho
        return

In [2]:
r = Rock(2300, 2500)
print(r)

## Exercises

- Add a name to the Rock — let the user pass it in. Hint: edit the `__init__()` class.
- Add an `elastic_impedance()` method to the class. Use `bruges.rockphysics.elastic_impedance()`.
- Make `acoustic_impedance()` a **property** of the instance, instead of a method. This requires a decorator.
- Add docstrings to the class itself, and to the (non-dunder) methods/properties.
- Add doctests to the methods/properties.
- Add a `__repr__()` method to control the way the rock displays. E.g. try this:
      def __repr__(self):
          return "Rock({})".format(self.Vp)
- Add a **class method** called `from_csv()` that makes a Rock from strings like "Sandstone,2300,1200,2500"
- Implement [`total_ordering`](https://docs.python.org/2/library/functools.html#functools.total_ordering) based on acoustic impedance."
----

## Solutions

In [3]:
import numpy as np
from bruges.rockphysics import elastic_impedance
from functools import total_ordering


@total_ordering
class Rock:
    """
    A class to hold rock properties.
    """
    
    def __init__(self, vp, vs=None, rho=None, name=None):
        """
        Example
        >>> r = Rock(2300, 1200, 2500, name="Rock Y")
        """
        self.vp = float(vp)
        self.vs = float(vs) if vs is not None else np.nan
        self.rho = float(rho) if rho is not None else np.nan
        self.name = str(name)
        return

    # Required for total ordering.
    def __eq__(self, other):
        return self.acoustic_impedance == other.acoustic_impedance

    # Required for total ordering.
    def __gt__(self, other):
        return self.acoustic_impedance > other.acoustic_impedance

    def __repr__(self):
        """
        Example
        >>> r = Rock(2300, 1200, 2500, name="Rock Y")
        >>> print(r)
        Rock("Rock Y": 2300.0, 1200.0, 2500.0)
        """
        return f"Rock(\"{self.name}\": {self.vp}, {self.vs}, {self.rho})"
    
    @property
    def acoustic_impedance(self):
        """
        Compute the acoustic impedance.
        
        Example
        >>> r = Rock(2300, rho=2500)
        >>> r.acoustic_impedance
        5750000.0
        """
        return self.vp * self.rho
    
    @classmethod
    def from_csv(cls, text):
        """
        Make a Rock object from a CSV-like string.

        Example
        >>> r = Rock.from_csv(",2300,,2500")
        >>> r.acoustic_impedance
        5750000.0

        """
        data = text.split(',')
        vp, vs, rho = [float(n) if n else np.nan for n in data[1:]]
        name = data[0] or None
        return cls(vp, vs, rho, name)

    def elastic_impedance(self, theta=0):
        """
        Compute the elastic impedance using `bruges`.
        
        Example
        >>> r = Rock(2300, 1200, 2500)
        >>> r.elastic_impedance(15)
        2013737.0019058161
        """
        return elastic_impedance(self.vp, self.vs, self.rho, theta)


## Example outputs

These are the kinds of output your class should give:

    >>> r = Rock(2300, 1200, 2500, name="Rock Y")
    
    >>> print(r)
    Rock("Rock Y": 2300, 1200, 2500)
    
    >>> Rock.from_csv("2300,1200,2500,Sandstone")
    Rock("Sandstone": 2300.0, 1200.0, 2500.0)
    
    >>> r.acoustic_impedance
    5750000
    
    >>> r.elastic_impedance(15)
    2013737.0019058161
    
    >>> import doctest
    >>> doctest.testmod(extraglobs={'r': Rock(2300, 1200, 2500, name="Rock Y")})
    TestResults(failed=0, attempted=3)


In [4]:
import doctest

doctest.testmod()

TestResults(failed=0, attempted=9)

In [5]:
r = Rock(2300, 1200, 2500, name="Rock Y")

In [6]:
print(r)

Rock("Rock Y": 2300.0, 1200.0, 2500.0)


In [7]:
Rock.from_csv("Sandstone,2300,1200,2500").acoustic_impedance

5750000.0

In [8]:
r.acoustic_impedance

5750000.0

In [9]:
r.elastic_impedance(15)

2013737.0019058161

In [10]:
Rock(2300, rho=2500) >= Rock(2100, rho=2500)

True

In [11]:
sorted([Rock(2300, rho=2500), Rock(2100, rho=2400), Rock(2000, rho=2600)])

[Rock("None": 2100.0, nan, 2400.0),
 Rock("None": 2000.0, nan, 2600.0),
 Rock("None": 2300.0, nan, 2500.0)]

## Implement your own class

- Core
- Core sample (SWC, SCAL)
- Well log
- Well (borehole)
- Seismic shot record
- Seismic trace (or line, or vol)
- Seismic survey plan
- Synthetic seismogram
- Well test or production log
- Basin or play
- Stratigraphic column
- Formation (top pick, map)
- Prospect or field
- Well report
- Seismic processing report
- Outcrop

----

### CORE

Likely mostly a data class. A core class might want to record the following attributes:

- ID or box number
- Top: 2300 m
- Base: 2350 m
- Diameter: 4.5 in
- Sections: 3 ft
- Condition: Fair
- Cut type (onethird, twothird, biscuit, uncut)
- Lithologies (as a {depth: lithology} dictionary perhaps)
- A set of core photographs
- A CAT scan 3D image
- A list of tests or analyses (these could be another class)
- Observations, eg grain-size curve, bioturbation, etc.

Note that it probbly would not reference 'parental' things like well name, instead belonging to a separate well object.

Calculated properties might include: 

- Length

Methods might include: 

- Make a table representation for Jupyter
- Plot a composite view
- Calculate texture from core photo
- Predict lithology from logs or core photo

----

### PROSPECT

See the Volumetrics notebook from the Fundamentals class notebooks. Attributes might include:

- Area, thickness, geometric factor, porosity, Bo or Bg, net:gross, saturation
- Distribution types for all those attributes
- Name, location, formation name, play type, and so on
- Probabilities: reservoir, trap, seal, migration

Properties:

- Gross rock volume
- Hydrocarbon pore volume
- Hydrocarbons in place
- Probability of discovery

Methods:

- Deterministic volumetrics
- Simulate properies using Monte Carlo
- Fit a distribution to a dataset (see Volumetrics notebook)
- Combine two prospects with dependent or independent risks

----

### WELL REPORT

Attributes:

- Serial number(s)
- Author, title, date
- Length
- Format
- Abstract
- File (eg PDF) location
- Physical location

Methods:

- Find a word or phrase
- Make a word cloud
- Summarize or get tags
- Get a particular page from the PDF
- Try to extract a table from a given page
- Run sentiment analysis
- Find figures, or particular types of figure
- Perform latent semantic analyis (LSA)

NLP tools are easy enough to find, but it might be easier to start with some web APIs, eg on rapidapi.com. For example, see this notebook: https://github.com/kwinkunks/notebooks/blob/master/Text_processing.ipynb 

There is an example of LSA in this tutorial: https://github.com/seg/2017-tle-hall

----

© 2019 Agile Scientific